In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/LA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1115657202,LA,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,1623197104,LA,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,4241517309,LA,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,5102977301,LA,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,5378948703,LA,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1115657202,LA,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,1623197104,LA,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,4241517309,LA,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,5102977301,LA,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,5378948703,LA,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
295,7205207001,LA,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,NaN
600,4082007104,LA,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,NaN
1003,1231277105,LA,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,NaN
1123,6556928303,LA,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,NaN
1317,5071027000,LA,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,NaN
...,...,...,...,...,...,...
12924,1824047101,LA,"208 VAN NESS AVE, WELSH, LA",34.070802,-118.315212,NaN
13154,4754577409,LA,"1307 MEEKS ST, BASTROP, LA",32.765897,-91.898381,NaN
13173,4115567105,LA,"1503 N 19TH ST, MONROE, LA",42.593950,-89.640270,NaN
13337,3211388504,LA,"405 E University Ave, Lafayette, LA",34.364883,-89.530010,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('22')] #LA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/63 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

63


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9334/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
295,7205207001,LA,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,[]
600,4082007104,LA,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,[]
1003,1231277105,LA,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,[]
1123,6556928303,LA,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,[]
1317,5071027000,LA,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,[]
...,...,...,...,...,...,...
12924,1824047101,LA,"208 VAN NESS AVE, WELSH, LA",34.070802,-118.315212,[]
13154,4754577409,LA,"1307 MEEKS ST, BASTROP, LA",32.765897,-91.898381,[220679505004018]
13173,4115567105,LA,"1503 N 19TH ST, MONROE, LA",42.593950,-89.640270,[]
13337,3211388504,LA,"405 E University Ave, Lafayette, LA",34.364883,-89.530010,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9334/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
295,7205207001,LA,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,NaN
600,4082007104,LA,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,NaN
1003,1231277105,LA,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,NaN
1123,6556928303,LA,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,NaN
1317,5071027000,LA,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,NaN
...,...,...,...,...,...,...
12924,1824047101,LA,"208 VAN NESS AVE, WELSH, LA",34.070802,-118.315212,NaN
13154,4754577409,LA,"1307 MEEKS ST, BASTROP, LA",32.765897,-91.898381,220679505004018
13173,4115567105,LA,"1503 N 19TH ST, MONROE, LA",42.593950,-89.640270,NaN
13337,3211388504,LA,"405 E University Ave, Lafayette, LA",34.364883,-89.530010,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
295,7205207001,LA,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,NaN
600,4082007104,LA,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,NaN
1003,1231277105,LA,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,NaN
1123,6556928303,LA,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,NaN
1317,5071027000,LA,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,NaN
1437,2674807701,LA,"40201 HIGHWAY 190 E, SLIDELL, LA",31.239600,-99.091648,NaN
1685,5622717101,LA,"107 HIGHWAY 90 W, NEW IBERIA, LA",30.045167,-94.887227,NaN
1686,6516038302,LA,"107 Highway 90 W, New Iberia, LA",30.045167,-94.887227,NaN
1796,6772877005,LA,"1505 N 19TH ST, MONROE, LA",42.593950,-89.640240,NaN
1869,7870737007,LA,"324 E LOCKWOOD ST, COVINGTON, LA",39.080982,-84.516600,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
295,7205207001,NaN
600,4082007104,NaN
1003,1231277105,NaN
1123,6556928303,NaN
1317,5071027000,NaN
...,...,...
12924,1824047101,NaN
13154,4754577409,220679505004018
13173,4115567105,NaN
13337,3211388504,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/LA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,382875,1115657202,LA,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,382876,1623197104,LA,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,382877,4241517309,LA,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,382878,5102977301,LA,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,382879,5378948703,LA,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,382875,1115657202,LA,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14,NaN
1,382876,1623197104,LA,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14,NaN
2,382877,4241517309,LA,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14,NaN
3,382878,5102977301,LA,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14,NaN
4,382879,5378948703,LA,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14,NaN
...,...,...,...,...,...,...,...,...
13664,396539,9651717708,LA,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,2.201501e+14,NaN
13665,396540,9659377206,LA,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,2.207100e+14,NaN
13666,396541,9719897702,LA,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,2.210304e+14,NaN
13667,396542,9806418300,LA,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,2.205500e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,382875,1115657202,LA,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14,NaN,220510205062018.0
1,382876,1623197104,LA,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14,NaN,220710134004025.0
2,382877,4241517309,LA,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14,NaN,220510242023039.0
3,382878,5102977301,LA,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14,NaN,220330040212030.0
4,382879,5378948703,LA,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14,NaN,220710139001017.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1115657202,LA,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,220510205062018.0
1,1623197104,LA,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,220710134004025.0
2,4241517309,LA,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,220510242023039.0
3,5102977301,LA,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,220330040212030.0
4,5378948703,LA,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,220710139001017.0
...,...,...,...,...,...,...
13664,9651717708,LA,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,220150111193014.0
13665,9659377206,LA,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,220710036001007.0
13666,9719897702,LA,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,221030406081008.0
13667,9806418300,LA,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,220550014178011.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
295,7205207001,LA,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,NaN
600,4082007104,LA,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,NaN
1003,1231277105,LA,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,NaN
1123,6556928303,LA,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,NaN
1317,5071027000,LA,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,NaN
1437,2674807701,LA,"40201 HIGHWAY 190 E, SLIDELL, LA",31.239600,-99.091648,NaN
1685,5622717101,LA,"107 HIGHWAY 90 W, NEW IBERIA, LA",30.045167,-94.887227,NaN
1686,6516038302,LA,"107 Highway 90 W, New Iberia, LA",30.045167,-94.887227,NaN
1796,6772877005,LA,"1505 N 19TH ST, MONROE, LA",42.593950,-89.640240,NaN
1869,7870737007,LA,"324 E LOCKWOOD ST, COVINGTON, LA",39.080982,-84.516600,NaN


Convert to File

In [19]:
fips_LA = fips_merge_drop

In [20]:
fips_LA.to_csv("../../../data/state_data/geo/geo_fips/23/LA_fips_scraped.csv") 